
# <font color=blue><div align="center">ST7 SNCF : Jalon 1</div></font>

## <font color=blue><div align="center">"<em>Indian Railways</em>"</div></font>

## Modules

In [147]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
%matplotlib inline
from dateutil import parser

# Module relatif à Gurobi
from gurobipy import *


# Module csv
import csv
from datetime import datetime

In [148]:
mini_instance="mini_instance.xlsx"
instance_simple="instance_WPY_simple.xlsx"
instance_realiste="instance_WPY_realiste_jalon1.xlsx"
test="mini_instance_test.xlsx"

## Traitement de données

In [253]:
def excel_to_dict(file :str)-> dict[str]:
    '''
    Takes excel file name in the repo
    and returns a dictionary[sheet_name]=Dataframe
    '''
    # Load the Excel file
    xl = pd.ExcelFile(file)
    
    # Get the names of the sheets
    sheet_names = xl.sheet_names
    
    # Initialize a dictionary to store the data
    data_dict = {}
    
    # Iterate over each sheet
    for sheet_name in sheet_names:
        # Read the sheet into a pandas DataFrame
        data_dict[sheet_name] = pd.read_excel(file, sheet_name=sheet_name)
    return data_dict

INSTANCE= excel_to_dict(instance_realiste)


In [254]:
indice=instance_realiste

### Dictionnaires utiles 

Trains de l'arrivée et du départs

In [255]:
trains_arrivee=[]
print(type(INSTANCE['Sillons arrivee']["JARR"])) #str
print(type(INSTANCE['Sillons depart']['JDEP'])) #timestamp
for index, row in INSTANCE["Sillons arrivee"].iterrows():
    id_train = row['n°TRAIN'],row['JARR']
    trains_arrivee.append(id_train)

trains_depart=[]

for index, row in INSTANCE["Sillons depart"].iterrows():
    numero_train = row['n°TRAIN'],row['JDEP']
    trains_depart.append(numero_train)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [256]:
print(trains_depart)
print(trains_arrivee)

[(400006, '09/08/2022'), (400016, '09/08/2022'), (400110, '09/08/2022'), (431035, '09/08/2022'), (431091, '09/08/2022'), (44203, '09/08/2022'), (44241, '09/08/2022'), (44249, '09/08/2022'), (44806, '09/08/2022'), (47245, '09/08/2022'), (47289, '09/08/2022'), (47821, '09/08/2022'), (489447, '09/08/2022'), (52672, '09/08/2022'), (54003, '09/08/2022'), (54051, '09/08/2022'), (54086, '09/08/2022'), (54231, '09/08/2022'), (450226, '10/08/2022'), (400014, '10/08/2022'), (412124, '10/08/2022'), (41258, '10/08/2022'), (431035, '10/08/2022'), (431091, '10/08/2022'), (44203, '10/08/2022'), (44241, '10/08/2022'), (44249, '10/08/2022'), (47205, '10/08/2022'), (48215, '10/08/2022'), (489044, '10/08/2022'), (52672, '10/08/2022'), (54003, '10/08/2022'), (54053, '10/08/2022'), (54231, '10/08/2022'), (71210, '10/08/2022'), (400006, '11/08/2022'), (400010, '11/08/2022'), (400016, '11/08/2022'), (431035, '11/08/2022'), (44203, '11/08/2022'), (44241, '11/08/2022'), (44806, '11/08/2022'), (450226, '11/08/2

In [5]:
print(trains_arrivee)

['sillon1', 'sillon2', 'sillon3']


Correspondance

In [257]:
def trains_depart_relatifs_a(train_depart)->list:
    '''
    Prend l'id d'un train du départ 
    et retourne une liste des id des trains
    d'arrivée qui le constituent
    '''
    trains_arrivee_associes = []
    for index, row in INSTANCE["Correspondances"].iterrows():
        if (row['n°Train depart'],row['Jour depart'])==train_depart :
            trains_arrivee_associes.append((row['n°Train arrivee'],row['Jour arrivee']))
        
    return trains_arrivee_associes

création des créneaux

In [283]:
def numero_jour_semaine(date_str):
    date = pd.to_datetime(date_str, format='%d/%m/%Y')
    numero_jour = date.dayofweek
    return numero_jour

def HARR_to_creneau_in_h(heure,indice):
    heure_str=''
    if indice==mini_instance:
        heure_str = heure.strftime("%H:%M")
    else:
        heure_str=heure
    heure1,minute= int(heure_str.split(':')[0]), int(heure_str.split(':')[1])    
    return heure1*4 + minute//15

In [281]:
print(HARR_to_creneau_in_h('17:46',instance_simple))

72
63


In [287]:
jour0=str(INSTANCE["Sillons arrivee"]["JARR"][0])
print('jour0: ',jour0)
indice_jour0=numero_jour_semaine(jour0)
# print(indice_jour0)

def creneaux_arrivee(jour0,INSTANCE,indice):
    creneaux_arrivee={}
    for index, row in INSTANCE["Sillons arrivee"].iterrows():
        delta=(datetime.strptime(row['JARR'],'%d/%m/%Y')-datetime.strptime(jour0,'%d/%m/%Y')).days
        creneaux_arrivee[(row['n°TRAIN'],row['JARR'])]=(delta//7)*96+numero_jour_semaine(row['JARR'])*96 + HARR_to_creneau_in_h(row['HARR'],indice)
    
    return creneaux_arrivee
    
def creneaux_depart(jour0,INSTANCE,indice):
    creneaux_depart={}
    for index, row in INSTANCE["Sillons depart"].iterrows():
        # ti=datetime.fromtimestamp(row['JDEP']))
        # print(row['JDEP'].to_pydatetime())
        time_str=row['JDEP']
        date=datetime.strptime('2002/09/23','%Y/%m/%d')
        if indice==mini_instance:
            time_str=row['JDEP'].strftime('%Y-%m-%d')
            date=datetime.strptime(time_str,'%Y-%m-%d')
        elif indice==instance_simple:
            time_str=row['JDEP']
            date=datetime.strptime(time_str,'%d/%m/%Y')
        else: 
            date=datetime.strptime(time_str,'%d/%m/%Y')
        delta=(date-datetime.strptime(jour0,'%d/%m/%Y')).days
        creneaux_depart[(row['n°TRAIN'],row['JDEP'])]=(delta//7)*96*7+numero_jour_semaine(row['JDEP'])*96 + HARR_to_creneau_in_h(row['HDEP'],indice)
        
    return creneaux_depart

print('creneaux_depart: ', creneaux_depart(jour0,INSTANCE,indice))
print('crenaux_arrivee: ',creneaux_arrivee(jour0,INSTANCE,indice))



jour0:  08/08/2022
creneaux_depart:  {(400006, '09/08/2022'): 159, (400016, '09/08/2022'): 187, (400110, '09/08/2022'): 179, (431035, '09/08/2022'): 156, (431091, '09/08/2022'): 121, (44203, '09/08/2022'): 167, (44241, '09/08/2022'): 149, (44249, '09/08/2022'): 108, (44806, '09/08/2022'): 128, (47245, '09/08/2022'): 153, (47289, '09/08/2022'): 140, (47821, '09/08/2022'): 185, (489447, '09/08/2022'): 120, (52672, '09/08/2022'): 160, (54003, '09/08/2022'): 111, (54051, '09/08/2022'): 118, (54086, '09/08/2022'): 111, (54231, '09/08/2022'): 152, (450226, '10/08/2022'): 221, (400014, '10/08/2022'): 271, (412124, '10/08/2022'): 204, (41258, '10/08/2022'): 202, (431035, '10/08/2022'): 252, (431091, '10/08/2022'): 217, (44203, '10/08/2022'): 263, (44241, '10/08/2022'): 245, (44249, '10/08/2022'): 204, (47205, '10/08/2022'): 248, (48215, '10/08/2022'): 256, (489044, '10/08/2022'): 200, (52672, '10/08/2022'): 256, (54003, '10/08/2022'): 207, (54053, '10/08/2022'): 254, (54231, '10/08/2022'): 248

Créneaux de l'arrivée et du départ

In [289]:
creneau_arrivee=creneaux_arrivee(jour0,INSTANCE,indice)
creneau_depart=creneaux_depart(jour0,INSTANCE,indice)
nombre_creneaux=96*8
if indice==instance_realiste:
    nombre_creneaux=96*10

Indisponibilités

In [ ]:
def creneaux_de_periode(indispo_str,nombre_creneaux): #jour, plage_horaire
    plage_jours=nombre_creneaux//96
    creneaux = []
    if indispo_str!='0':
        periodes = indispo_str.split(';')
        for periode in periodes:
            jour, plage_horaire = periode.strip('()').split(',')
            jour = int(jour)
            jours=[jour+7*k for k in range(0,plage_jours//7+1)]
            plage_horaire = plage_horaire.strip('()').split('-')
            heure_debut, minute_debut = map(int, plage_horaire[0].split(':'))
            creneau_debut=minute_debut//15+1
            heure_fin, minute_fin = map(int, plage_horaire[1].split(':'))
            creneau_fin=minute_fin//15+1


            if (heure_debut,minute_debut)>=(heure_fin,minute_fin):
                for jour1 in jours:
                    for c in range((jour1-1)*96+heure_debut*4+creneau_debut,jour1*96):
                        if c<=nombre_creneaux:
                            creneaux.append(c)
                    for c in range(jour1*96,jour1*96+heure_fin*4+creneau_fin):
                        if c<=nombre_creneaux: creneaux.append(c)
            else:
                for jour1 in jours:
                    for c in range((jour1-1)*96+heure_debut*4+creneau_debut,(jour1-1)*96+heure_fin*4+creneau_fin):
                        if c<=nombre_creneaux: creneaux.append(c) 
    return creneaux

# data1 = INSTANCE1['Machines']
data = INSTANCE['Machines']

df = pd.DataFrame(data)

df['Indisponibilites']=df['Indisponibilites'].astype(str)
# Apply the function to each row
df['Indisponibilites_TimeSlots'] = df['Indisponibilites'].apply(creneaux_de_periode,nombre_creneaux=nombre_creneaux)


DEB_INDIS = df.loc[df['Machine'] == 'DEB', 'Indisponibilites_TimeSlots'].values[0]
FOR_INDIS = df.loc[df['Machine'] == 'FOR', 'Indisponibilites_TimeSlots'].values[0]
DEG_INDIS = df.loc[df['Machine'] == 'DEG', 'Indisponibilites_TimeSlots'].values[0]

## Modèle mathématique 
On a modélisé notre problème en partant du fait que les machines ne sont disponibles que pour un seul train et dans un seul créneau. \
De ce fait, on introduit les variables *binaires* suivantes : \
\
    - $(deb_{ijc})$ : variable binaire indiquant si la machine de débranchement est utilisée \
    pour le sillion d'arrivée d'identifiant $i$ dans le jour $j$ au créneau $c$ \
    - $(for_{ijc})$ : variable binaire indiquant si la machine de formation est utilisée \
    pour le sillion du départ d'identifiant $i$ dans le jour $j$ au créneau $c$ \
    - $(deg_{ijc})$ : variable binaire indiquant si la machine de dégarage est utilisée \
    pour le sillion du départ d'identifiant $i$ dans le jour $j$ au créneau $c$

Les créneaux ont une durée de $15 min$ correspondant à la durée d'une tâche machine. Les valeurs possibles des créneaux dépendent de l'instance, \
soit alors $c \in \{1,2,..., c_{max}\} $ où $c=1$ correspond au premier créneau de 15 min à parfir du premier jour de travail à $0:00$ et \
$c_{max}$ le dernier créneau du dernier jours de travail à partir de $23:45$.

On note ainsi : 
$$A = \{\text{Les identifiants des trains d'arrivée}\}$$
$$D = \{\text{Les identifiants des trains du départ}\}$$
$$C= \{1,2,..., c_{max}\}$$

In [290]:
# IMPLEMNTATION DES VARIABLES 
m= Model('Modèle')
deb = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
form = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
deg = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deg_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}

## Contraintes
### 1. Unique débranchement, formation et dégarage
Chaque train d'arrivé subit un seul et unique débranchement, chaque train du départ subit un seul et unique débranchement et dégarage. 
$$ \forall i \in A \quad \sum_{c} deb_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} for_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} deg_{i,c} = 1 $$

In [291]:
uniqueDeb_train = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeb_train{i}') for i in trains_arrivee}
uniqueform_train = {i : m.addConstr(quicksum([form[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueFor_train{i}') for i in trains_depart}
uniqueDeg_train = {i : m.addConstr(quicksum([deg[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeg_train{i}') for i in trains_depart}

### 2. Une seule tâche machine par créneau  
Chaque machine est exploitée par au plus un seul train sur chaque créneau.
$$ \forall c \in C \quad \sum_{i \in A} deb_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} for_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} deg_{i,c} \leq 1 $$

In [292]:
uniqueDeb_creneau = {c : m.addConstr(quicksum([deb[(i, c)] for i in trains_arrivee]) <=1, name = f'UniqueDeb_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueFor_creneau = {c : m.addConstr(quicksum([form[(i, c)] for i in trains_depart]) <=1, name = f'UniqueFor_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueDeg_creneau = {c : m.addConstr(quicksum([deg[(i, c)] for i in trains_depart ]) <=1, name = f'UniqueDeg_creneau{c}') for c in range(1, nombre_creneaux + 1)}


### 3. Débranchement se fait après l'arrivée du train
Pour chaque train d'arrivée $i$, le débranchement ne peut pas avoir lieu dans les créneaux précedent celui d'arrivée $c^{a}_{i}$ après une heure de tâches humaines.
$$ \forall i \in A \quad \sum_{c \space \leq \space c^{a}_{i}+4} deb_{i,c} = 0$$ 

In [293]:
deb_apres_arrivee = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, creneau_arrivee[i] + 5)]) ==0, name = f'deb_apres_arrivee{i}') for i in trains_arrivee}


KeyError: ('sillon1', 769)

### 4. Formation d'un train de départ 
Pour chaque train du départ $t$, la formation ne peut avoir lieu qu'après débranchement de tous les trains ayant les wagons qui le constituent. \
On note : $$ \forall t \in D \quad A_t = \{\text{Les identifiants des trains d'arrivée ayant au moins un wagon du train t}\}$$
On peut exprimer la contrainte comme suit : 
$$ \forall t \in D, \space\forall c \in C, \space\forall i \in A_t \quad for_{t,c} \leq \sum_{c' \space \lt \space c} deb_{i,c'}$$ 

In [318]:
for_apres_deb = {(t,c,i) : m.addConstr(form[(t,c)]<=quicksum([deb[(i, c_p)] for c_p in range(1, c)]), name = f'for_apres_deb{t}{c}{i}') for t in trains_depart for c in range(1,nombre_creneaux+1) for i in trains_depart_relatifs_a(t)}


### 5. Dégarage après formation
Pour chaque train du départ, le dégarage ne peut avoir lieu qu'après la formation du train et $150$ minutes soit $10$ créneaux de tâches humaines.
$$ \forall t \in D, \space\forall c \in C \quad deg_{t,c} \leq \sum_{c' \space \lt \space c-10} for_{t,c'}$$ 

In [314]:
deg_apres_for = {(t,c) : m.addConstr(deg[(t,c)]<=quicksum([form[(t, c_p)] for c_p in range(1, c-9)]), name = f'deg_apres_for{t},{c}') for t in trains_depart for c in range(1,nombre_creneaux+1)}


### 6. Dégarage avant départ
Pour chaque train du départ $t$, le dégarage ne peut pas avoir lieu dans les créneaux suivants \
celui de $20$ minutes (de tâches humaines) avant le créneau du départ $c^{d}_{t}$.
$$ \forall t \in D \quad \sum_{c \space \geq \space c^{d}_{t}-2} deg_{t,c} = 0$$ 

In [313]:
deg_avant_depart = {t : m.addConstr(quicksum([deg[(t, c)] for c in range(creneau_depart[t]-2, nombre_creneaux+1)])==0, name = f'depart_apres_deg{t}') for t in trains_depart}


### 7. Disponibilités des machines
Il y a des créneaux où les machines sont indisponible, soient alors  $ \space C_{deb}, C_{for}, C_{deg} \space$  ces crénaux d'indisponibilités.
$$ \forall i \in A \quad \sum_{c \space \in \space C_{deb}} deb_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{for}} for_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{deg}} deg_{i,c} = 0 $$

In [312]:
indisponibilite_deb = {i : m.addConstr(quicksum([deb[(i, c)] for c in DEB_INDIS]) ==0, name = f'indisponibilité_deb{i}') for i in trains_arrivee }
indisponibilite_for = {i : m.addConstr(quicksum([form[(i, c)] for c in FOR_INDIS]) ==0, name = f'indisponibilité_for{i}') for i in trains_depart}
indisponibilite_deg = {i : m.addConstr(quicksum([deg[(i, c)] for c in DEG_INDIS]) ==0, name = f'indisponibilité_deg{i}') for i in trains_depart}


## Résolution

In [319]:
objfunction = 1
m.setObjective(objfunction , GRB.MINIMIZE)
m.params.outputflag = 1
m.update()

In [320]:
m.params.outputflag = 0

In [325]:
m.setParam(GRB.Param.IterationLimit, 3600)
m.optimize()
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print(m.display(), "\n\tN'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print(m.display(), "\n\tEST NON BORNÉ!!!")

GurobiError: Out of memory

In [247]:
for train in trains_arrivee:
    print(train)
    print('--------------------------------------------')
    for c in range(1,nombre_creneaux+1):
        if deb[(train,c)].x==1:
            print('deb de {} se fait au créneau : {}, l arrivee de ce train est au creneau {} '.format(train,c,creneau_arrivee[train]))
            print('-------------------------------------------------------')


for train in trains_depart:
    print(train)
    print('--------------------------------------------')
    for c in range(1,nombre_creneaux+1):
        if form[(train,c)].x==1:
            print('For de {} se fait au créneau: {}, le depart de ce train se fait à {}'.format(train,c,creneau_depart[train]))
            print('-------------------------------------------------------')
        if deg[(train,c)].x==1:
            print('deg de {} se fait au créneau: {}, le depart de ce train se fait à {} '.format(train,c,creneau_depart[train]))
            print('-------------------------------------------------------')
        


('A1', '08/08/2022')
--------------------------------------------
deb de ('A1', '08/08/2022') se fait au créneau : 54, l arrivee de ce train est au creneau 40 
-------------------------------------------------------
('A3', '08/08/2022')
--------------------------------------------
deb de ('A3', '08/08/2022') se fait au créneau : 93, l arrivee de ce train est au creneau 88 
-------------------------------------------------------
('A2', '08/08/2022')
--------------------------------------------
deb de ('A2', '08/08/2022') se fait au créneau : 53, l arrivee de ce train est au creneau 46 
-------------------------------------------------------
('A2', '09/08/2022')
--------------------------------------------
deb de ('A2', '09/08/2022') se fait au créneau : 147, l arrivee de ce train est au creneau 142 
-------------------------------------------------------
('A3', '09/08/2022')
--------------------------------------------
deb de ('A3', '09/08/2022') se fait au créneau : 189, l arrivee de c

In [ ]:
print('deb de {} se fait au créneau : {}'.format(2,7))

deb de 2 se fait au créneau : 7


# Questions 
- Horraires du départ non conforme avec les créneaux machines
- Créneaux et jours ? indices dépendants 
- Il ya un choix à faire lors de la correspondance entre tâche humaine et créneaux où \
entre tâche machine et créneaux.